In [ ]:
##### Create 200 m buffer around barrier polygons (to make sure that all nodes in the barriers have elevation data) #####
#########################################################################################################################

import os
import osmnx as ox
import geopandas as gpd
import networkx as nx

# Loop within polygons folder and create a 200m buffer to increase the size of each barrier.  
rootdir = 'E:\\Barriers\\Polygons'
extensions = ('.shp')

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        ext = os.path.splitext(file)[-1].lower()
        if ext in extensions:
            file_path = os.path.join(subdir, file)
            name= file.replace(".shp","")
            print(name)
            # Read polygons
            barrier = gpd.read_file(file_path)
            # Change crs to projected
            barrier = barrier.to_crs(epsg=32119)
            # Buffer 100m
            barrier2 = barrier['geometry'] = barrier.geometry.buffer(200)
            # Reproject to original crs
            barrier2= barrier2.to_crs(epsg=4326)
            # Save file
            barrier2.to_file("E:\\Barriers\\Buffers_200m\\{0}.shp".format(name))

In [ ]:
##### Check if raster and NC barrier (200m buffer) overlap, if true, clip raster using polygon and save it #####
################################################################################################################

from osgeo import gdal, ogr
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.mask import mask
from shapely.geometry import box
import geopandas as gpd
from fiona.crs import from_epsg
import pycrs
import os

def getFeatures(gdf):
    """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

for filename1 in os.listdir('E:\\Barriers\\CUDEM'):
    if filename1.endswith('.tif'):
        raster_dir=('E:\\Barriers\\CUDEM\\{0}'.format(filename1))
        raster_name=filename1.replace (".tif", "")
        raster = gdal.Open(raster_dir)
        #print(raster_dir)
        # Get raster geometry
        transform = raster.GetGeoTransform()
        pixelWidth = transform[1]
        pixelHeight = transform[5]
        cols = raster.RasterXSize
        rows = raster.RasterYSize

        xLeft = transform[0]
        yTop = transform[3]
        xRight = xLeft+cols*pixelWidth
        yBottom = yTop+rows*pixelHeight

        ring = ogr.Geometry(ogr.wkbLinearRing)
        ring.AddPoint(xLeft, yTop)
        ring.AddPoint(xLeft, yBottom)
        ring.AddPoint(xRight, yBottom)
        ring.AddPoint(xRight, yTop)
        ring.AddPoint(xLeft, yTop)
        rasterGeometry = ogr.Geometry(ogr.wkbPolygon)
        rasterGeometry.AddGeometry(ring)
        
        for filename2 in os.listdir('E:\\Barriers\\Buffers_200m'):
            if filename2.endswith('.shp'):
                vector_dir=('E:\\Barriers\\Buffers_200m\\{0}'.format(filename2))
                vector_name=filename2.replace(".shp", "")
                vector = ogr.Open(vector_dir)
                #print(vector_dir)
                        
                # Get vector geometry
                layer = vector.GetLayer()
                feature = layer.GetFeature(0)
                vectorGeometry = feature.GetGeometryRef()

                #print(rasterGeometry.Intersect(vectorGeometry))
                
                if  rasterGeometry.Intersect(vectorGeometry) == True:
                        print('True')
                        # Output raster
                        out_tif = os.path.join('E:\\Barriers\\CUDEM_CLIP\\{0}_{1}.tif'.format(vector_name,raster_name))
                        # Read the data
                        data = rasterio.open(raster_dir)
                        barrier = gpd.read_file(vector_dir)
                        # Project the Polygon into same CRS as the grid
                        barrier = barrier.to_crs(crs=data.crs.data)
                        coords = getFeatures(barrier)
                        # Clip the raster with Polygon
                        out_img, out_transform = mask(dataset=data, shapes=coords, crop=True)
                        # Copy the metadata
                        out_meta = data.meta.copy()
                        # Parse EPSG code
                        #epsg_code = int(data.crs.data['init'][5:])
                        out_meta.update({"driver": "GTiff",
                                        "height": out_img.shape[1],
                                        "width": out_img.shape[2],
                                        "transform": out_transform,
                                        "crs": "EPSG:4269"}
                                                 )
                        with rasterio.open(out_tif, "w", **out_meta) as dest:
                            dest.write(out_img)
                        print(out_tif)
                            
                else:
                    continue

In [ ]:
##### With the clipped rasters, create CUDEM mosaic for each barrier #####
#########################################################################

# Import packages
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
import glob
import os
import matplotlib.pyplot as plt
%matplotlib inline

for vector in os.listdir(r'E:\Barriers\Buffers_200m'):
            if vector.endswith('.shp'):
                vector_name= vector.replace('.shp', '')
                vector_name= vector_name + "_"
                # List for the source files
                src_files_to_mosaic = []
                for raster in os.listdir(r'E:\Barriers\CUDEM_Clip'):
                            if raster.startswith(vector_name):
                                src = rasterio.open(r'E:\Barriers\CUDEM_Clip\{0}'.format(raster))
                                src_files_to_mosaic.append(src)
                                # Merge function returns a single mosaic array and the transformation info
                                mosaic, out_trans = merge(src_files_to_mosaic)
                                # Copy the metadata
                                out_meta = src.meta.copy()
                                # Update the metadata
                                out_meta.update({"driver": "GTiff",
                                                 "height": mosaic.shape[1],
                                                 "width": mosaic.shape[2],
                                                 "transform": out_trans,
                                                 "crs": "EPSG:4269"
                                                 }
                                                )

                                # Write the mosaic raster to disk
                                with rasterio.open(r"E:\Barriers\CUDEM_Mosaic\{0}.tif".format(vector_name), "w", **out_meta) as dest:
                                    dest.write(mosaic)
                                print(vector_name)
                            else:
                                continue
                                           


In [ ]:
##### With CUDEM mosaics, extract elevation data for each node #####
####################################################################

%matplotlib inline
import osmnx as ox
import os
import matplotlib.pyplot as plt
import geopandas as gpd
import rasterio
import numpy as np


# loop through mosaics
for filename1 in os.listdir(r'E:\Barriers\CUDEM_Mosaic'):
    if filename1.endswith('.tif'):
        raster_dir=(r'E:\Barriers\CUDEM_Mosaic\{0}'.format(filename1))
        name=filename1.replace(".tif", "")
        print(name)
        # Open raster
        raster= rasterio.open(raster_dir)
        try:
            # Read polygon
            poly = gpd.read_file(r'E:\Barriers\Polygons\{0}.shp'.format(name))
            # Extract just the geometry (shapely object) part
            poly_geo = poly['geometry'].iloc[0]
            # Clean it with a buffer
            poly_geo = poly_geo.buffer(0)
            poly_geo.is_valid
            G = ox.graph_from_polygon(poly_geo, network_type='drive', simplify=True, clean_periphery=True)
            # Retrieve nodes and edges
            nodes, edges = ox.graph_to_gdfs(G)
            # Create an index for the geodataframe nodes
            nodes['index'] = range(0, len(nodes))
            # Convert to raster crs 
            nodes = nodes.to_crs("epsg:4269") 

            # Extract elevations for each point (node) in the network using CUDEM raster
            values=[]
            for point in nodes['geometry']:
                x = point.xy[0][0]
                y = point.xy[1][0]
                row, col=raster.index(x,y)
                value=raster.read(1)[row,col]
                values.append(value)   

            # Add elevation to nodes geodataframe as a new column and save shapefiles
            nodes['elevation']=values
            nodes.to_file(r"E:\Barriers\Nodes\{0}.shp".format(name))
            edges.to_file(r"E:\Barriers\Edges\{0}.shp".format(name))
        except:
            continue

        


In [ ]:
##### Calculate exceedance probabilities for all node elevations #####
######################################################################


import os
import geopandas as gpd
import pandas as pd
import numpy as np

rootdir = r'E:\Barriers\Nodes'
extensions = ('.shp')

# Loop through files and open barrier graphml
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        ext = os.path.splitext(file)[-1].lower()
        if ext in extensions:
            file_path = os.path.join(subdir, file)
            barrier = file.replace(".shp","")
            print(barrier)
            nodes = gpd.read_file(file_path)
            exceed = pd.read_csv("E:\Barriers\Exceedance\Exceedance_prob\{0}_Exceedance.csv".format(barrier), sep=",", header=0)
            for i in range(0,len(nodes)):
                node_elev= float(nodes.elevation[i])
                exceed_x= exceed.MaxWL
                exceed_y= exceed.Return_Pd
                exceedance= np.interp(node_elev, exceed_x, exceed_y)
                print(exceedance)
                nodes.at[i,'exceedance']=exceedance
                nodes.to_file("E:\Barriers\Exceedance\Exceedance_nodes\{0}.shp".format(barrier))

In [ ]:
##### Extract networks using OSMnx and add elevation as a node attribute #####
##############################################################################


import os
import osmnx as ox
import geopandas as gpd
import networkx as nx

# Loop in barriers folder (original polygons, not buffers)
rootdir = r'E:\Barriers\Polygons'
extensions = ('.shp')

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        ext = os.path.splitext(file)[-1].lower()
        if ext in extensions:
            print (file)
            file_path = os.path.join(subdir, file)
            
            try:

                # Read polygons, extract geometry (shapely object), and clean it with buffer
                poly = gpd.read_file(file_path)
                poly_geo = poly['geometry'].iloc[0]
                poly_geo = poly_geo.buffer(0)
            
                # Pull the roads with OSMnx
                G = ox.graph_from_polygon(poly_geo, network_type='drive', simplify=True, clean_periphery=True)
                
                # Give to each node a new index based on integers from 0 and then add the osmid as an attribute
                osmids = list(G.nodes)
                G = nx.relabel.convert_node_labels_to_integers(G)
                osmid_values = {k:v for k, v in zip(G.nodes, osmids)}
                nx.set_node_attributes(G, osmid_values, 'osmid')

                # Open nodes.shp with elevation data and add as an attribute to the nodes in the graph
                shp= gpd.read_file(r'E:\Barriers\Exceedance\Exceedance_nodes\{0}'.format(file))
                nx.set_node_attributes(G, shp.elevation, 'Elevations')
                nx.set_node_attributes(G, shp.exceedance, 'Exceedance')

                # Save street network as graphml
                name=file.replace('.shp','')
                ox.save_graphml(G, filepath=r'E:\Barriers\Graphs\{0}.graphml'.format(name))
                
            except:
                print('error')
                continue
        

In [ ]:
##### Analysis of connected components and GCS behavior when nodes are removed #####
########### Calculation of Robustness as defined in Schneider (2011) ###############
####################################################################################

import os
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import geopandas as gpd
import osmnx as ox
import networkx as nx
import pandas as pd
import numpy as np
import contextily as ctx
import statistics


##### GCS behavior #####


rootdir = 'E:\Barriers\Graphs'
extensions = ('.graphml')

# Loop through files and open barrier graphml
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        ext = os.path.splitext(file)[-1].lower()
        if ext in extensions:
            file_path = os.path.join(subdir, file)
            barrier = file.replace(".graphml","")
            print(barrier)
            G = nx.read_graphml(file_path)
            if len(G.nodes(data=True))>100:
                GCCs=[]
                # Pull out elevation attribute
                E = nx.get_node_attributes(G,'Elevations')
                # Convert str values in float to be able to sort them 
                E = dict(zip(E.keys(), [float(value) for value in E.values()]))
                # Sort it based on elevation, min first
                Sorted_E = sorted(E.items(), key=lambda item: item[1])
                CCs = np.zeros([len(Sorted_E),2])
                # Select first element of each tuple in the list (nodes ID):
                FT = [i[0] for i in Sorted_E]
                # Select second element of each tuple in the list (elevation) and convert to float
                ST = [i[1] for i in Sorted_E]
                for i in range(len(ST)):
                    ST[i] = float(ST[i])
                # Loop through all nodes
                for i in range(0, len(FT)):
                    # Find the node with lowest elevation from the list using i and remove it
                    G.remove_nodes_from(FT[0:i])
                    # Find the number of connected components and its respective size
                    GCC = [len(c)
                            for c in sorted(nx.weakly_connected_components(G), key=len, reverse=True)
                        ]
                    GCCs.append(GCC) # list with the number of connected components and its size
                    # Fill array, first column corresponds to FGC (first giant component), second column to SGC (second giant component)
                    if len(GCC)==1:
                        CCs[int(i),0]=GCC[0]/len(FT)
                        CCs[int(i),1]=0
                    else:
                        CCs[int(i),0]=GCC[0]/len(FT)
                        CCs[int(i),1]=GCC[1]/len(FT)
                # Find the node that, once removed, the FGC decreases and the SGC reaches its maximum
                m = max(CCs[:,1])
                pos=[i for i, j in enumerate(CCs[:,1]) if j == m]
                pos= pos[0] # position of max value in SGC
                critical= pos-1 # position of the critical node whose removal causes the percolation transition.
                elev=ST[critical] # find elevation of the critical node
                removed=pos # number of nodes removed when percolation threshold occurs

    #             # Calculate robustness following Schneider's equation (2011) and include it in plot
    #             s= sum(CCs[:,0])
    #             r= s/len(FT)
    #             r_norm= int(round(r/0.5*100))


                # Plot
                col1=[] 
                for i in range(0,len(FT)):
                    if i==critical:
                        col1.append('#D53032') 
                    else:
                        col1.append('#000000')  
                col2=[]
                for i in range(0,len(FT)):
                    if i==critical:
                        col2.append('#D53032') 
                    else:
                        col2.append('#808080') 
                col3=[]
                for i in range(0,len(FT)):
                    if i==critical:
                        col3.append('#D53032') 
                    else:
                        col3.append('#9ACD32') 

                f, (ax1,ax2) = plt.subplots(2, 1, sharex=True)

                x_coord = 1 * np.arange(len(FT))/len(FT) # Fraction of nodes removed
                ax1.plot(x_coord, CCs[:,0],':ok')
                for i in range (len(FT)):
                    ax1.plot(x_coord[i],CCs[i,0],'o', markersize=5, color=col1[i]) # plot GCS with two colors to highlight critical node
                ax1.set_ylabel("First Giant Component Size")
                ax3 = ax1.twinx()
                ax3.plot(x_coord, CCs[:,1],':ok')
                for i in range (len(FT)):
                    ax3.plot(x_coord[i],CCs[i,1],'o', markersize=5, color=col2[i]) # plot GCS with two colors to highlight target node
                ax3.set_ylabel("Second Giant Component Size")
                ax2.plot(x_coord,CCs[:,0],':ok')
                for i in range (len(FT)):
                    ax2.plot(x_coord[i],CCs[i,0],'o', markersize=5, color=col1[i]) # plot GCS with two colors to highlight target node
                ax2.set_ylabel("First Giant Component Size")
                ax4 = ax2.twinx()
                ax4.plot(x_coord,ST,':o', color='#9ACD32')
                for i in range (len(FT)):
                    ax4.plot(x_coord[i],ST[i],'o', markersize=5, color=col3[i]) # plot GCS with two colors to highlight target node
                ax2.set_ylabel("First Giant Component Size",)
                ax4.set_ylabel("Elevation")
                ax2.set_xlabel("Fraction of removed nodes") 
                legend_elements1 = [Line2D([0], [0], marker='o', color='#000000', label='FGC', markersize=10),
                                    Line2D([0], [0], marker='o', color='#808080', label='SGC', markersize=10),
                                    Line2D([0], [0], marker='o', color='#D53032', label='Critical node', markersize=10)]
                ax1.legend(handles=legend_elements1, loc="best", frameon=False, fontsize=18)
                legend_elements2 = [Line2D([0], [0], marker ='o', color='#000000', label='FGC', markersize=10),
                                   Line2D([0], [0], marker='o', color='#9ACD32', label='Elevation', markersize=10)]
                ax1.legend(handles=legend_elements1, loc="best", frameon=False, fontsize=18)

                plt.show()
                plt.rcParams["font.size"]= 20
                plt.rcParams["figure.figsize"] = (15,15)

                f.savefig("E:\\Barriers\\Figures\\GCS_individual\\{0}.png".format(barrier), dpi=500, facecolor='w')

####################### Static maps for each network using OSM as basemap #########################################################
###################################################################################################################################
            
                # Read polygons
                poly = gpd.read_file("E:\\Barriers\\Polygons\\{0}.shp".format(barrier))
                # Extract just the geometry (shapely object) part
                poly_geo = poly['geometry'].iloc[0]
                # Clean it with a buffer
                poly_geo = poly_geo.buffer(0)
                poly_geo.is_valid
                graph = ox.graph_from_polygon(poly_geo, network_type='drive', simplify=True, clean_periphery=True)
                # Plot network
                ox.plot_graph(ox.project_graph(graph))
                # Retrieve nodes and edges as geodataframes
                nodes, edges = ox.graph_to_gdfs(graph)
                # Create an index for the geodataframe nodes
                nodes['index'] = range(0, len(nodes))

                # Convert E dict in pandas dataframe and name columns
                E = pd.DataFrame(list(E.items()),columns = ['index','elevation'])
                # Convert all columns in numerics so there are no errors when merging
                E = E.apply(pd.to_numeric)
                # Join pandas dataframe to nodes geodataframe using 'index' so that the gdf has elevation
                nodes = nodes.merge(E, on='index')

                # Create new columns for color and size
                def color(row):
                    if row['elevation'] < elev:
                        val = "black"
                    elif row['elevation']== elev:
                        val = "red"
                    else:
                        val = "green"
                    return val

                def size(row):
                    if row['elevation'] == elev:
                        val = 50
                    else:
                        val = 30
                    return val

                nodes['Color'] = nodes.apply(color, axis=1) # New column with color categories 
                nodes['Size'] = nodes.apply(size, axis=1) # New column with size categories

                # Create map with OSM as basemap
                fig, ax = plt.subplots()
                nodes = nodes.to_crs(epsg=3857) # convert to EPSG used by basemaps
                edges = edges.to_crs(epsg=3857)
                nodes.plot(ax=ax, color=nodes.Color, markersize=nodes.Size, zorder=2, legend=True) # plot nodes
                edges.plot(ax=ax, alpha=0.2, color='black', zorder=1) # plot edges
                ctx.add_basemap(ax, zoom=13, source=ctx.providers.OpenStreetMap.Mapnik) # add basemap (OSM)
                plt.xticks(fontsize=12) # reduce fontsize of x axis
                plt.yticks(fontsize=12) # reduce fontsize of y axis
                legend_elements = [Line2D([0], [0], marker='o', color='black', label='Connected nodes',
                                          markerfacecolor='g', markersize=10),
                                   Line2D([0], [0], marker='o', color='black', label='Disconnected nodes',
                                          markerfacecolor='b', markersize=10),
                                   Line2D([0], [0], marker='o', color='black', label='Target node',
                                          markerfacecolor='r', markersize=10),
                                   ] # create legend
                ax.legend(handles=legend_elements, loc='best', frameon=False)
                ax.set_title(barrier, fontsize=22)
                ax.ticklabel_format(style='plain')         
                plt.rcParams["figure.figsize"] = (25,25)
                plt.savefig('E:\\Barriers\\Figures\\Maps\\{0}).png'.format(barrier), dpi=300, facecolor='w')   
            else:
                print("<100")
                continue

In [ ]:
#### Create table with information for each barrier #####
#########################################################

import os
import geopandas as gpd
import osmnx as ox
import networkx as nx
import pandas as pd
import numpy as np
import statistics
from scipy import stats


barriers=[]
n_nodes=[]
robustness=[]
mean_elevation=[]
median_elevation=[]
elevation_red=[]
exceedance_red=[]
removed_nodes=[]
removed_perc=[]
FGC_before=[]
FGC_after=[]
SGC_before=[]
SGC_after=[]
threshold=[]
entropy=[]

# area_curve=[]


entropy_bins = 36
def reverse_bearing(x):
    return x + 180 if x < 180 else x - 180
def count_and_merge(n, bearings):
    n = n * 2
    bins = np.arange(n + 1) * 360 / n
    count, _ = np.histogram(bearings, bins=bins)
    count = np.roll(count, 1)
    return count[::2] + count[1::2]

rootdir = 'E:\\Barriers\\Graphs'
extensions = ('.graphml')

# Loop through files and open barrier graphml
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        ext = os.path.splitext(file)[-1].lower()
        if ext in extensions:
            file_path = os.path.join(subdir, file)
            barrier = file.replace(".graphml","")
            G = nx.read_graphml(file_path)
            if len(G.nodes(data=True))>100:
                GCCs=[]
                N= len(G.nodes(data=True))
                print(barrier)
                barriers.append(barrier)
                n_nodes.append(N)
                # Pull out elevation attribute
                E = nx.get_node_attributes(G,'Elevations')
                # Convert str values in float to be able to sort them 
                E = dict(zip(E.keys(), [float(value) for value in E.values()]))
                # Sort it based on elevation, min first
                Sorted_E = sorted(E.items(), key=lambda item: item[1])
                CCs = np.zeros([len(Sorted_E),2])
                # Select first element of each tuple in the list (nodes ID):
                FT = [i[0] for i in Sorted_E]
                # Select second element of each tuple in the list (elevation) and convert to float
                ST = [i[1] for i in Sorted_E]
                for i in range(len(ST)):
                    ST[i] = float(ST[i])

                # Calculate elevation stats    
                mean_elev = statistics.mean(ST)
                mean_elevation.append(mean_elev)
                median_elev = statistics.median(ST)
                median_elevation.append(median_elev)
                

                # Remove nodes by elevation and calculate size of first and second component
                for i in range(0, len(FT)):
                    # Find the node with lowest elevation from the list using i and remove it
                    G.remove_nodes_from(FT[0:i])
                    # Find the number of connected components and its respective size
                    GCC = [len(c)
                            for c in sorted(nx.weakly_connected_components(G), key=len, reverse=True)
                        ]
                    GCCs.append(GCC) # list with the number of connected components and its size
                    # Fill array, first column corresponds to FGC (first giant component), second column to SGC (second giant component)
                    if len(GCC)==1:
                        CCs[int(i),0]=GCC[0]/len(FT)
                        CCs[int(i),1]=0
                    else:
                        CCs[int(i),0]=GCC[0]/len(FT)
                        CCs[int(i),1]=GCC[1]/len(FT)

                # Find the node that, once removed, the FGC decreases and the SGC reaches its maximum (critical percolation threshold)
                m = max(CCs[:,1])
                pos=[i for i, j in enumerate(CCs[:,1]) if j == m]
                pos= pos[0] # position of max value in SGC
                critical= pos-1 # position of the critical node whose removal causes the percolation transition.
                elev=ST[critical] # find elevation of the critical node
                elevation_red.append(elev)
                removed=pos # number of nodes removed when percolation threshold occurs
                removed_nodes.append(removed)
                perc_removed=int(removed)/N*100
                removed_perc.append(perc_removed)
                try:
                    FGC_bf=GCCs[critical][0]
                    FGC_before.append(FGC_bf)
                except IndexError:
                    FGC_before.append(0)
                try:
                    FGC_af=GCCs[pos][0]
                    FGC_after.append(FGC_af)
                except IndexError:
                    FGC_after.append(0)
                try:
                    SGC_bf= GCCs[critical][1]
                    SGC_before.append(SGC_bf)
                except IndexError:
                    SGC_before.append(0)
                try:
                    SGC_af=GCCs[pos][1]
                    SGC_after.append(SGC_af)
                except IndexError:
                    SGC_after.append(0)
                x_coord = 1 * np.arange(len(FT))/len(FT) # Fraction of nodes removed
                thresh= x_coord[critical]
                threshold.append(thresh)

                # Exceedance
                Sorted_E = pd.DataFrame(Sorted_E)
                exceed = pd.read_csv("E:\Barriers\Exceedance\Exceedance_prob\{0}_Exceedance.csv".format(barrier), sep=",", header=0)
                exceed_x= exceed.MaxWL
                exceed_y= exceed.Return_Pd
                node_elev= elev
                exceedance= np.interp(node_elev, exceed_x, exceed_y)
                exceedance_red.append(exceedance)

                # Calculate robustness following Schneider's equation (2011) and include it in plot
                s= sum(CCs[:,0])
                r= s/len(FT)
                r_norm= int(round(r/0.5*100))
                robustness.append(r_norm)


                # Calculate entropy
                poly = gpd.read_file(r'E:\Barriers\Polygons\{0}.shp'.format(barrier))
                poly_geo = poly['geometry'].iloc[0]
                poly_geo = poly_geo.buffer(0)
                G = ox.graph_from_polygon(poly_geo, network_type='drive', simplify=True, clean_periphery=True)
                Gu = ox.add_edge_bearings(ox.get_undirected(G))
                b = pd.Series([d['bearing'] for u, v, k, d in Gu.edges(keys=True, data=True)])
                bearings = pd.concat([b, b.map(reverse_bearing)]).reset_index(drop='True')
                bin_counts = count_and_merge(entropy_bins, bearings)
                orientation_entropy = stats.entropy(bin_counts)
                perfect_grid = [1] * 4 + [0] * (entropy_bins - 4)
                min_entropy = stats.entropy(perfect_grid)
                max_entropy = np.log(entropy_bins)
                orientation_order = 1 - ((orientation_entropy - min_entropy) / (max_entropy - min_entropy)) ** 2
                entropy.append(orientation_order)




    #             # Calculate area under curve in plot GCS-Elevation, where x is elevation and y GSC
    #             x=np.array(ST)
    #             y=CCs
    #             area = np.sum((x[1:] - x[:-1]) * (y[1:] + y[:-1]) / 2)
    #             area_curve.append(area)
    #             print(area)
            else:
                print("<100")
                continue
table = list(zip(barriers,n_nodes,robustness,mean_elevation,median_elevation,elevation_red,exceedance_red, removed_nodes,removed_perc, FGC_before, FGC_after, SGC_before, SGC_after, threshold, entropy))
table = pd.DataFrame(table, columns=['Barrier','Number_nodes','Robustness_%','Mean_elevation','Median_elevation','Elevation_red_node','Exceedance_red_node','Removed_nodes','Removed_%','FGC_before','FGC_after','SGC_before','SGC_after','Critical_threshold','Entropy'])
table.to_csv('E:\\Barriers\\Table2.csv')

In [ ]:
##### Plot elevation (y axis) by fraction of nodes removed (x axis) - All barriers #####
########################################################################################


import os
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import geopandas as gpd
import osmnx as ox
import networkx as nx
import pandas as pd
import numpy as np


rootdir = 'E:\\Barriers\\Graphs'
extensions = ('.graphml')

# Loop through files and open barrier graphml
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        ext = os.path.splitext(file)[-1].lower()
        if ext in extensions:
            file_path = os.path.join(subdir, file)
            barrier = file.replace(".graphml","")
            print(barrier)
            G = nx.read_graphml(file_path)
            if len(G.nodes(data=True))>100:
                N= len(G.nodes(data=True))
                # Pull out elevation attribute
                E = nx.get_node_attributes(G,'Elevations')
                # Convert str values in float to be able to sort them 
                E = dict(zip(E.keys(), [float(value) for value in E.values()]))
                # Sort it based on elevation, min first
                Sorted_E = sorted(E.items(), key=lambda item: item[1])
                CCs = np.zeros([len(Sorted_E),1])
                # Select first element of each tuple in the list (nodes ID):
                FT = [i[0] for i in Sorted_E]
                # Select second element of each tuple in the list (elevation) and convert to float
                ST = [i[1] for i in Sorted_E]
                for i in range(len(ST)):
                    ST[i] = float(ST[i])

                # Loop through all nodes
                for i in range(0, len(FT)):
                    # Find the node with lowest elevation from the list using i and remove it
                    G.remove_nodes_from(FT[0:i])
                    # Check giant component size
                    CCs[int(i),0]= largest_cc = len(max(nx.weakly_connected_components(G), key=len))/len(FT)

                 # Calculate robustness following Schneider's equation (2011) and include it in plot
                s= sum(CCs[:,0])
                r= s/len(FT)
                r_norm= int(round(r/0.5*100))
                print(r_norm)

                def color():
                    if r_norm <= 25:
                        val = "#fff600"
                    elif r_norm >25 and r_norm<=50:
                        val = "#bddf26"
                    elif r_norm>50 and r_norm<=75:
                        val = '#29af7f'
                    elif r_norm>75 and r_norm<=100:
                        val = '#2e6f8e'
                    else:
                        val = 'black'
                    return val


                x_coord = 1 * np.arange(len(CCs))/len(CCs) # x axis
                color = color()
                plt.plot(x_coord,ST,'-', color=color, label=barrier)
                plt.rcParams["figure.figsize"] = (15,15)  
                plt.rcParams["font.size"]= 20            
            else:
                print("<100")
                continue
                
legend_elements = [Line2D([0], [0], color="#fff600", label='R <= 25%',
                                      markersize=10),
                              Line2D([0], [0], color="#bddf26", label='25% > R <= 50%'),
                              Line2D([0], [0], color='#29af7f', label='50% > R <= 75%'),
                              Line2D([0], [0],  color='#2e6f8e', label='75% > R <= 100%'),
                              ] # create legend

plt.legend(handles=legend_elements, loc='best', frameon=False)
plt.xlabel("Fraction of removed nodes")
plt.ylabel("Elevation")
plt.title("Elevation - All NC barriers", fontsize=22)

plt.savefig(r'E:\Barriers\Figures\GCS_All\Elevation_All.png', dpi=300, facecolor='w')
plt.show()

In [ ]:
##### Plot exceedance return periods (y axis) by fraction of nodes removed (x axis) - All barriers #####
########################################################################################################


import os
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import geopandas as gpd
import osmnx as ox
import networkx as nx
import pandas as pd
import numpy as np


rootdir = 'E:\\Barriers\\Graphs'
extensions = ('.graphml')

# Loop through files and open barrier graphml
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        ext = os.path.splitext(file)[-1].lower()
        if ext in extensions:
            file_path = os.path.join(subdir, file)
            barrier = file.replace(".graphml","")
            print(barrier)
            G = nx.read_graphml(file_path)
            N= len(G.nodes(data=True))
            
            if N>100:
                # Pull out elevation attribute
                E = nx.get_node_attributes(G,'Exceedance')
                # Convert str values in float to be able to sort them 
                E = dict(zip(E.keys(), [float(value) for value in E.values()]))
                # Sort it based on elevation, min first
                Sorted_E = sorted(E.items(), key=lambda item: item[1])
                CCs = np.zeros([len(Sorted_E),1])
                # Select first element of each tuple in the list (nodes ID):
                FT = [i[0] for i in Sorted_E]
                # Select second element of each tuple in the list (elevation) and convert to float
                ST = [i[1] for i in Sorted_E]
                for i in range(len(ST)):
                    ST[i] = float(ST[i])

                # Loop through all nodes
                for i in range(0, len(FT)):
                    # Find the node with lowest elevation from the list using i and remove it
                    G.remove_nodes_from(FT[0:i])
                    # Check giant component size
                    CCs[int(i),0]= largest_cc = len(max(nx.weakly_connected_components(G), key=len))/len(FT)

                 # Calculate robustness following Schneider's equation (2011) and include it in plot
                s= sum(CCs[:,0])
                r= s/len(FT)
                r_norm= int(round(r/0.5*100))
                print(r_norm)

                def color():
                    if r_norm <= 25:
                        val = "#fff600"
                    elif r_norm >25 and r_norm<=50:
                        val = "#bddf26"
                    elif r_norm>50 and r_norm<=75:
                        val = '#29af7f'
                    elif r_norm>75 and r_norm<=100:
                        val = '#2e6f8e'
                    else:
                        val = 'black'
                    return val


                x_coord = 1 * np.arange(len(CCs))/len(CCs) # x axis
                color = color()
                plt.plot(x_coord,ST,'-', color=color, label=barrier)
                plt.rcParams["figure.figsize"] = (15,15)  
                plt.rcParams["font.size"]= 20       
            else:
                print("<100")
                continue

legend_elements = [Line2D([0], [0], color="#fff600", label='R <= 25%',
                                      markersize=10),
                              Line2D([0], [0], color="#bddf26", label='25% > R <= 50%'),
                              Line2D([0], [0], color='#29af7f', label='50% > R <= 75%'),
                              Line2D([0], [0],  color='#2e6f8e', label='75% > R <= 100%'),
                              ] # create legend
                
plt.legend(handles=legend_elements, loc='best', frameon=False)
plt.xlabel("Fraction of removed nodes")
plt.ylabel("Exceedance - Return Period")
plt.title("Exceedance - All NC barriers", fontsize=22)

plt.savefig(r'E:\Barriers\Figures\GCS_All\Exceedance_All.png', dpi=300, facecolor='w')
plt.show()

In [ ]:
##### Plot GCS behavior (y axis) by fraction of nodes removed (x axis) - All barriers #####
###########################################################################################

import os
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import geopandas as gpd
import osmnx as ox
import networkx as nx
import pandas as pd
import numpy as np


rootdir = 'E:\\Barriers\\Graphs'
extensions = ('.graphml')

# Loop through files and open barrier graphml
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        ext = os.path.splitext(file)[-1].lower()
        if ext in extensions:
            file_path = os.path.join(subdir, file)
            barrier = file.replace(".graphml","")
            print(barrier)
            G = nx.read_graphml(file_path)
            N= len(G.nodes(data=True))
            
            if N>100:
                # Pull out elevation attribute
                E = nx.get_node_attributes(G,'Elevations')
                # Convert str values in float to be able to sort them 
                E = dict(zip(E.keys(), [float(value) for value in E.values()]))
                # Sort it based on elevation, min first
                Sorted_E = sorted(E.items(), key=lambda item: item[1])
                CCs = np.zeros([len(Sorted_E),1])
                # Select first element of each tuple in the list (nodes ID):
                FT = [i[0] for i in Sorted_E]
                # Select second element of each tuple in the list (elevation) and convert to float
                ST = [i[1] for i in Sorted_E]
                for i in range(len(ST)):
                    ST[i] = float(ST[i])

                # Loop through all nodes
                for i in range(0, len(FT)):
                    # Find the node with lowest elevation from the list using i and remove it
                    G.remove_nodes_from(FT[0:i])
                    # Check giant component size
                    CCs[int(i),0]= largest_cc = len(max(nx.weakly_connected_components(G), key=len))/len(FT)

                # Calculate robustness following Schneider's equation (2011) and include it in plot
                s= sum(CCs[:,0])
                r= s/len(FT)
                r_norm= int(round(r/0.5*100))
                print(r_norm)

                def color():
                    if r_norm <= 25:
                        val = "#fff600"
                    elif r_norm >25 and r_norm<=50:
                        val = "#bddf26"
                    elif r_norm>50 and r_norm<=75:
                        val = '#29af7f'
                    elif r_norm>75 and r_norm<=100:
                        val = '#2e6f8e'
                    else:
                        val = 'black'
                    return val


                x_coord = 1 * np.arange(len(CCs))/len(CCs) # x axis
                color = color()
                plt.plot(x_coord,CCs[:,0],'-', color=color, label=barrier)
                plt.rcParams["figure.figsize"] = (15,15)  
                plt.rcParams["font.size"]= 20       
            else:
                print("<100")
                continue

legend_elements = [Line2D([0], [0], color="#fff600", label='R <= 25%',
                                      markersize=10),
                              Line2D([0], [0], color="#bddf26", label='25% > R <= 50%'),
                              Line2D([0], [0], color='#29af7f', label='50% > R <= 75%'),
                              Line2D([0], [0],  color='#2e6f8e', label='75% > R <= 100%'),
                              ] # create legend

plt.legend(handles=legend_elements, loc='best', frameon=False)
plt.xlabel("Fraction of removed nodes")
plt.ylabel("GCS")
plt.title("GCS - All NC barriers", fontsize=22)


plt.savefig(r'E:\Barriers\Figures\GCS_all\GCS_All.png', dpi=300, facecolor='w')
plt.show()

In [ ]:
##### Plot GCS behavior (y axis) by elevation (x axis) - All barriers #####
###########################################################################

import os
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import geopandas as gpd
import osmnx as ox
import networkx as nx
import pandas as pd
import numpy as np

rootdir = 'E:\\Barriers\\Graphs'
extensions = ('.graphml')

# Loop through files and open barrier graphml
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        ext = os.path.splitext(file)[-1].lower()
        if ext in extensions:
            file_path = os.path.join(subdir, file)
            barrier = file.replace(".graphml","")
            print(barrier)
            G = nx.read_graphml(file_path)
            N= len(G.nodes(data=True))
            if N>100:
                # Pull out elevation attribute
                E = nx.get_node_attributes(G,'Elevations')
                # Convert str values in float to be able to sort them 
                E = dict(zip(E.keys(), [float(value) for value in E.values()]))
                # Sort it based on elevation, min first
                Sorted_E = sorted(E.items(), key=lambda item: item[1])
                CCs = np.zeros([len(Sorted_E),1])
                # Select first element of each tuple in the list (nodes ID):
                FT = [i[0] for i in Sorted_E]
                # Select second element of each tuple in the list (elevation) and convert to float
                ST = [i[1] for i in Sorted_E]
                for i in range(len(ST)):
                    ST[i] = float(ST[i])

                # Loop through all nodes
                for i in range(0, len(FT)):
                    # Find the node with lowest elevation from the list using i and remove it
                    G.remove_nodes_from(FT[0:i])
                    # Check giant component size
                    CCs[int(i),0]= largest_cc = len(max(nx.weakly_connected_components(G), key=len))/len(FT)

                # Calculate robustness following Schneider's equation (2011) and include it in plot
                s= sum(CCs[:,0])
                r= s/len(FT)
                r_norm= int(round(r/0.5*100))
                print(r_norm)

                def color():
                    if r_norm <= 25:
                        val = "#fff600"
                    elif r_norm >25 and r_norm<=50:
                        val = "#bddf26"
                    elif r_norm>50 and r_norm<=75:
                        val = '#29af7f'
                    elif r_norm>75 and r_norm<=100:
                        val = '#2e6f8e'
                    else:
                        val = 'black'
                    return val



                plt.plot(ST,CCs,'-', color=color(), label=barrier)
                plt.rcParams["figure.figsize"] = (15,15)  
                plt.rcParams["font.size"]= 20      
            
            else:
                print("<100")
                continue


legend_elements = [Line2D([0], [0], color="#fff600", label='R <= 25%',
                                      markersize=10),
                              Line2D([0], [0], color="#bddf26", label='25% > R <= 50%'),
                              Line2D([0], [0], color='#29af7f', label='50% > R <= 75%'),
                              Line2D([0], [0],  color='#2e6f8e', label='75% > R <= 100%'),
                              ] # create legend


plt.legend(handles=legend_elements, loc='best', frameon=False)
plt.xlabel("Elevation")
plt.ylabel("GCS")
plt.title("Elevation vs GCS - All NC barriers", fontsize=22)


plt.savefig(r'E:\Barriers\Figures\GCS_all\GCS_Elevation_All.png', dpi=300, facecolor='w')
plt.show()

In [ ]:
##### Plot GCS behavior (y axis) by exceedance (x axis) - All barriers #####
###########################################################################

import os
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import geopandas as gpd
import osmnx as ox
import networkx as nx
import pandas as pd
import numpy as np

rootdir = 'E:\\Barriers\\Graphs'
extensions = ('.graphml')

# Loop through files and open barrier graphml
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        ext = os.path.splitext(file)[-1].lower()
        if ext in extensions:
            file_path = os.path.join(subdir, file)
            barrier = file.replace(".graphml","")
            print(barrier)
            G = nx.read_graphml(file_path)
            N= len(G.nodes(data=True))
            if N>100:
            
                # Pull out elevation attribute
                E = nx.get_node_attributes(G,'Exceedance')
                # Convert str values in float to be able to sort them 
                E = dict(zip(E.keys(), [float(value) for value in E.values()]))
                # Sort it based on elevation, min first
                Sorted_E = sorted(E.items(), key=lambda item: item[1])
                CCs = np.zeros([len(Sorted_E),1])
                # Select first element of each tuple in the list (nodes ID):
                FT = [i[0] for i in Sorted_E]
                # Select second element of each tuple in the list (elevation) and convert to float
                ST = [i[1] for i in Sorted_E]
                for i in range(len(ST)):
                    ST[i] = float(ST[i])

                # Loop through all nodes
                for i in range(0, len(FT)):
                    # Find the node with lowest elevation from the list using i and remove it
                    G.remove_nodes_from(FT[0:i])
                    # Check giant component size
                    CCs[int(i),0]= largest_cc = len(max(nx.weakly_connected_components(G), key=len))/len(FT)

                # Calculate robustness following Schneider's equation (2011) and include it in plot
                s= sum(CCs[:,0])
                r= s/len(FT)
                r_norm= int(round(r/0.5*100))
                print(r_norm)

                def color():
                    if r_norm <= 25:
                        val = "#fff600"
                    elif r_norm >25 and r_norm<=50:
                        val = "#bddf26"
                    elif r_norm>50 and r_norm<=75:
                        val = '#29af7f'
                    elif r_norm>75 and r_norm<=100:
                        val = '#2e6f8e'
                    else:
                        val = 'black'
                    return val



                plt.plot(ST,CCs,'-', color=color(), label=barrier)
                plt.rcParams["figure.figsize"] = (15,15)  
                plt.rcParams["font.size"]= 20            
            else:
                print("<100")
                continue


legend_elements = [Line2D([0], [0], color="#fff600", label='R <= 25%',
                                      markersize=10),
                              Line2D([0], [0], color="#bddf26", label='25% > R <= 50%'),
                              Line2D([0], [0], color='#29af7f', label='50% > R <= 75%'),
                              Line2D([0], [0],  color='#2e6f8e', label='75% > R <= 100%'),
                              ] # create legend


plt.legend(handles=legend_elements, loc='best', frameon=False)
plt.xlabel("Exceedance - Return periods")
plt.ylabel("GCS")
plt.title("Exceedance vs GCS - All NC barriers", fontsize=22)
plt.set(xlim = (0,150))


plt.savefig(r'E:\Barriers\Figures\GCS_all\GCS_Exceedance_All.png', dpi=300, facecolor='w')
plt.show()

In [ ]:
##### Plot Rank - Elevations & Exceedance red node #####
########################################################


import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

fig, ax = plt.subplots()
ax.plot(rank,sorted_elevation, ':og')
ax1 = ax.twinx()
ax1.plot(rank,sorted_exceedance, ':ob')
plt.rcParams["figure.figsize"] = (15,15)  
plt.rcParams["font.size"]= 20 
plt.xlabel("Rank")
ax.set_xlabel("Rank")
ax.set_ylabel("Elevations - Critical node")
ax1.set_ylabel("Return periods - Critical node")


legend_elements = [Line2D([0], [0], marker='o', color='green', label='Elevation'),
                   Line2D([0], [0], marker='o', color='blue', label='Exceedance return period')] # create legend

ax.legend(handles=legend_elements, loc="best", frameon=False, fontsize=18)

In [ ]:
# ##### Analysis of connected components and GCS behavior when nodes are removed #####
# ########### Calculation of Robustness as defined in Schneider (2011) ###############
# ####################################################################################

# import os
# import matplotlib.pyplot as plt
# from matplotlib.lines import Line2D
# import geopandas as gpd
# import osmnx as ox
# import networkx as nx
# import pandas as pd
# import numpy as np
# import contextily as ctx
# import statistics


# ##### GCS behavior #####


# rootdir = 'E:\Barriers\Graphs'
# extensions = ('.graphml')

# # Loop through files and open barrier graphml
# for subdir, dirs, files in os.walk(rootdir):
#     for file in files:
#         ext = os.path.splitext(file)[-1].lower()
#         if ext in extensions:
#             file_path = os.path.join(subdir, file)
#             barrier = file.replace(".graphml","")
#             print(barrier)
#             G = nx.read_graphml(file_path)
                      
#             # Pull out elevation attribute
#             E = nx.get_node_attributes(G,'Elevations')
#             # Convert str values in float to be able to sort them 
#             E = dict(zip(E.keys(), [float(value) for value in E.values()]))
#             # Sort it based on elevation, min first
#             Sorted_E = sorted(E.items(), key=lambda item: item[1])
#             CCs = np.zeros([len(Sorted_E),2])
#             # Select first element of each tuple in the list (nodes ID):
#             FT = [i[0] for i in Sorted_E]
#             # Select second element of each tuple in the list (elevation) and convert to float
#             ST = [i[1] for i in Sorted_E]
#             for i in range(len(ST)):
#                 ST[i] = float(ST[i])
#             # Loop through all nodes
#             for i in range(0, len(FT)):
#                 # Find the node with lowest elevation from the list using i and remove it
#                 G.remove_nodes_from(FT[0:i])
#                 # Check number of connected components/clusters
#                 CCs[int(i),0] = nx.algorithms.components.number_weakly_connected_components(G)
#                 # Check giant component size
#                 CCs[int(i),1]= largest_cc = len(max(nx.weakly_connected_components(G), key=len))/len(FT)

#             # Check elevation of node that causes the highest drop in the GCS
#             Cliff = np.zeros([len(CCs),1])
#             N= len(Cliff)
#             GCS = CCs[:,1]
#             V1 = GCS[:-1]  # create vector with all but the last element 
#             V2 = GCS[1:]   # create vector with all but the first element
#             for i in range(0,len(V1)):
#                 Cliff[i,0]= V1[i]-V2[i] # calculate differences in GCS between the removal of one node and the next
#             m = max(Cliff) # max value in cliff corresponds to largest drop in the GCS
#             pos=[i for i, j in enumerate(Cliff) if j == m] # obtain position of node that causes the drop (top of the cliff)
#             pos=pos[0] # get the integer 
#             elev=ST[pos] # find elevation of the node that cracks the network
#             removed=pos+1 # number of nodes removed until reaching the drop (target node included)

#             # Calculate robustness following Schneider's equation (2011) and include it in plot
#             s= sum(CCs[:,1])
#             r= s/len(FT)
#             r_norm= int(round(r/0.5*100))

#             # Plot GCS behavior

#             col=[]
#             for i in range(0,len(CCs)):
#                 if i==pos:
#                     col.append('#D53032') 
#                 else:
#                     col.append('#4682B4')  

#             f, (ax1, ax2) = plt.subplots(2, 1, figsize=(15,15), sharex=True)
#             x_coord = 1 * np.arange(len(CCs))/len(CCs) # x axis
#             ax1.plot(x_coord,CCs[:,0],':ok') # plot number of connected components
#             ax1.set_ylabel("Number connected components")
#             for i in range (len(CCs)):
#                 ax2.plot(x_coord[i],CCs[i,1],'s', markersize= 10, color=col[i], linestyle='-') # plot GCS with two colors to highlight target node
#             ax3 = ax2.twinx()
#             ax3.plot(x_coord,ST,':o', markersize=10, color='#9ACD32', label='Elevation') # plot elevation in GCS plot
#             ax2.set_ylabel("Giant Component Size",)
#             ax3.set_ylabel("Elevation")
#             ax2.set_xlabel("Fraction of removed nodes") 
#             ax1.set_title(barrier, fontsize=22)

#             legend_elements = [Line2D([0], [0], marker='s', color='#4682B4', label='Nodes',
#                                       markersize=10),
#                                Line2D([0], [0], marker='s', color='#D53032', label='Target node',
#                                       markersize=10),
#                                Line2D([0], [0], marker='s', color='#9ACD32', label='Elevation',
#                                       markersize=10),
#                                ]
#             ax2.legend(handles=legend_elements, loc="lower left", bbox_to_anchor=(0.1, -0.4), frameon=False, fontsize=18)
#             ax2.annotate(r'R=%s'%(r_norm)+ '%', xy=(0.5,0.9), xycoords='axes fraction', fontsize=18)
#             plt.xticks(fontsize=12) # reduce fontsize of x axis
#             plt.yticks(fontsize=12) # reduce fontsize of y axis
#             f.tight_layout()
#             #plt.show()

#             f.savefig("E:\\Barriers\\Figures\\GCS\\{0}.png".format(barrier), dpi=500, facecolor='w')

# ####################### Static maps for each network using OSM as basemap #########################################################
# ###################################################################################################################################
            
#             # Read polygons
#             poly = gpd.read_file("E:\\Barriers\\Polygons\\{0}.shp".format(barrier))
#             # Extract just the geometry (shapely object) part
#             poly_geo = poly['geometry'].iloc[0]
#             # Clean it with a buffer
#             poly_geo = poly_geo.buffer(0)
#             poly_geo.is_valid
#             graph = ox.graph_from_polygon(poly_geo, network_type='drive', simplify=True, clean_periphery=True)
#             # Plot network
#             ox.plot_graph(ox.project_graph(graph))
#             # Retrieve nodes and edges as geodataframes
#             nodes, edges = ox.graph_to_gdfs(graph)
#             # Create an index for the geodataframe nodes
#             nodes['index'] = range(0, len(nodes))

#             # Convert E dict in pandas dataframe and name columns
#             E = pd.DataFrame(list(E.items()),columns = ['index','elevation'])
#             # Convert all columns in numerics so there are no errors when merging
#             E = E.apply(pd.to_numeric)
#             # Join pandas dataframe to nodes geodataframe using 'index' so that the gdf has elevation
#             nodes = nodes.merge(E, on='index')

#             # Create new columns for color and size
#             def color(row):
#                 if row['elevation'] < elev:
#                     val = "black"
#                 elif row['elevation']== elev:
#                     val = "red"
#                 else:
#                     val = "green"
#                 return val

#             def size(row):
#                 if row['elevation'] == elev:
#                     val = 50
#                 else:
#                     val = 30
#                 return val

#             nodes['Color'] = nodes.apply(color, axis=1) # New column with color categories 
#             nodes['Size'] = nodes.apply(size, axis=1) # New column with size categories

#             # Create map with OSM as basemap
#             fig, ax = plt.subplots()
#             nodes = nodes.to_crs(epsg=3857) # convert to EPSG used by basemaps
#             edges = edges.to_crs(epsg=3857)
#             nodes.plot(ax=ax, color=nodes.Color, markersize=nodes.Size, zorder=2, legend=True) # plot nodes
#             edges.plot(ax=ax, alpha=0.2, color='black', zorder=1) # plot edges
#             ctx.add_basemap(ax, zoom=13, source=ctx.providers.OpenStreetMap.Mapnik) # add basemap (OSM)
#             plt.xticks(fontsize=12) # reduce fontsize of x axis
#             plt.yticks(fontsize=12) # reduce fontsize of y axis
#             legend_elements = [Line2D([0], [0], marker='o', color='black', label='Connected nodes',
#                                       markerfacecolor='g', markersize=10),
#                                Line2D([0], [0], marker='o', color='black', label='Disconnected nodes',
#                                       markerfacecolor='b', markersize=10),
#                                Line2D([0], [0], marker='o', color='black', label='Target node',
#                                       markerfacecolor='r', markersize=10),
#                                ] # create legend
#             ax.legend(handles=legend_elements, loc='best', frameon=False)
#             ax.set_title(barrier, fontsize=22)
#             ax.ticklabel_format(style='plain')         
#             plt.rcParams["figure.figsize"] = (25,25)
#             plt.savefig('E:\\Barriers\\Figures\\Maps\\{0}.png'.format(barrier), dpi=300, facecolor='w')
              
